In [53]:
import numpy as np
import torch
from typing import Literal, Callable
from sklearn.metrics import classification_report


class LogisticRegressionRBF(torch.nn.Module):
    
    def __init__(self, x_basis: torch.Tensor, rbf: Literal['linear', 'gaussian'] = 'gaussian',
                 print_function: Callable = print):
        """
        :param x_basis: centers of basis functions
        :param rbf: type of rbf function. Available: ['linear', 'gaussian']
        """
        super(LogisticRegressionRBF, self).__init__()
        
        self.w = torch.nn.Linear(x_basis.shape[0], 1)
        self.rbf = rbf 
        self.x_basis = x_basis
        self.print = print_function
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x: torch.Tensor = None, phi_matrix: torch.Tensor = None) -> torch.Tensor:
        """
        Returns a "probability" (confidence) of class 1
        
        :param x: 2D array
        :param phi_matrix: 2D array
        :return: 1D array
        """
        if phi_matrix is None:
            phi_matrix = self.make_phi_matrix(x)
            
        return self.sigmoid(self.w(phi_matrix))

    def make_phi_matrix(self, x: torch.Tensor) -> torch.Tensor:
        """
        Returns k x n array with calculated phi(x_i, x_basis_j)
        
        :param x: Array k x m dimensional. k different x_i and m features
        """
        n = x.shape[0]
        m = x.shape[1]
        k = n * m  # Количество всех элементов

        repd_row_matrix = torch.tile(x, (1, n)).reshape(n ** 2, m)
        repd_whole_matrix =  torch.tile(x, (n, 1))
        phi = (repd_row_matrix - repd_whole_matrix) ** 2
        phi = phi.sum(axis=1).reshape(n, n)
        
        if self.rbf == 'linear':
            phi = phi ** 0.5
            phi = phi / phi.max()
        elif self.rbf == 'gaussian':
            phi = torch.exp(-phi)
        elif self.rbf == 'multiquadratic':
            phi = (1 + phi) ** 0.5
            phi = phi / phi.max()
            
        return phi
    
    def fit(self, x, y, epochs):
        
        print_epochs = np.unique(np.geomspace(1, epochs + 1, 15, dtype=int))
        
        phi_matrix = self.make_phi_matrix(x)
        optimizer = torch.optim.Adam(self.parameters())
        criterion = torch.nn.CrossEntropyLoss()
        loss = torch.nn.BCELoss()
        
        for epoch in range(1, epochs + 1):
            optimizer.zero_grad()
            output = loss(self.forward(x, phi_matrix).flatten(), y)
            output.backward()
            optimizer.step()
            
            with torch.no_grad():
                if epoch + 1 in print_epochs:
                    print(f'Epoch: {epoch: 5d} | CrossEntropyLoss: {output.item(): 0.5f}')
                
        return self

    def metrics_tab(self, x, y):
        y_prob = self.forward(x)
        y_pred = (y_prob > 0.5) * 1
        
        return self.print(classification_report(y, y_pred))

In [54]:
from sklearn.datasets import make_classification

x, y = make_classification(n_samples=500, n_features=4, random_state=1)
x = torch.FloatTensor(x[:, :2])
y = torch.FloatTensor(y)

In [55]:
model = LogisticRegressionRBF(x)

In [8]:
print(model.metrics_tab(x, y))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       251
         1.0       1.00      1.00      1.00       249

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



In [9]:
x

tensor([[ 2.6573,  0.0904, -0.4647,  ...,  0.7859,  0.5237,  0.2753],
        [ 0.7945, -1.5304,  0.1904,  ...,  1.3272, -0.8044, -0.7947],
        [-0.7132, -0.0342, -0.8635,  ..., -1.3989,  0.9047, -0.2815],
        ...,
        [ 0.6250,  1.2871,  0.1154,  ...,  0.2378,  0.0766, -1.5264],
        [ 0.9123,  0.3679, -1.0304,  ..., -1.2586,  1.5245,  0.5917],
        [ 0.7844, -0.0076, -1.2507,  ..., -1.2982,  1.5009,  0.3329]])